In [1]:
def evaluate(path_gold, path_test):
    gold_file = open(path_gold, encoding='utf-8', newline='\n')
    test_file = open(path_test, encoding='utf-8', newline='\n' if path_test.endswith('out.conll') else '\r\n')

    gold_word_id = ''
    
    n_total = 0
    n_labeled = 0
    n_unlabeled = 0
    
    n_virt_nodes_gold = 0
    n_virt_nodes_test = 0
    n_virt_nodes_labeled = 0
    n_virt_nodes_unlabeled = 0
    
    for test_line in test_file:

        if test_line == '\r\n':
            gold_word_id = ''
        else:
            test_fields = test_line[:-2].split('\t')
            test_word_id = test_fields[0]
            
            while gold_word_id != test_word_id:
                gold_line = gold_file.readline()
                gold_fields = gold_line[:-1].split('\t')
                gold_word_id = gold_fields[0]
            
            # print(test_fields)
            if test_fields == ['']:
                continue
            if test_fields[1] != 'unused' and (not test_fields[1].startswith('virt_') or not gold_fields[1].startswith('virt_') or test_fields[7] != 'unused' or gold_fields[7] != 'unused'):
            #if True:
                n_total += 1
                if test_fields[6] == gold_fields[6]:
                    n_unlabeled += 1 
                    
                    if test_fields[1].startswith('virt_'):
                        n_virt_nodes_unlabeled += 1

                    if test_fields[7] == gold_fields[7]:
                        n_labeled += 1

                        if test_fields[1].startswith('virt_'):
                            n_virt_nodes_labeled += 1
                        
                        
                if gold_fields[1].startswith('virt_') and gold_fields[7] != 'unused':
                    n_virt_nodes_gold += 1
                
                if test_fields[1].startswith('virt_') and test_fields[7] != 'unused':
                    n_virt_nodes_test += 1

    gold_file.close()
    test_file.close()

    return n_total, n_unlabeled, n_labeled, n_unlabeled/n_total, n_labeled/n_total, n_virt_nodes_gold, n_virt_nodes_test, n_virt_nodes_unlabeled, n_virt_nodes_labeled, n_virt_nodes_unlabeled/n_virt_nodes_test if n_virt_nodes_test > 0 else 0, n_virt_nodes_unlabeled/n_virt_nodes_gold if n_virt_nodes_gold > 0 else 0, n_virt_nodes_labeled/n_virt_nodes_test if n_virt_nodes_test > 0 else 0, n_virt_nodes_labeled/n_virt_nodes_gold if n_virt_nodes_gold > 0 else 0

train turbo:
    turbo_parser.exe --train --file_train=train.conll --file_model=zz.model
parse turbo:
    turbo_parser.exe --test --evaluate --file_model=zz.model --file_test=test.conll --file_prediction=test.conll.predicted

In [3]:
pretty_result = lambda x: f'''1. Всего слов - {x[0]}\n2. Кол-во слов с правильно предсказанной вершиной - {x[1]}\n3. Кол-во слов с правильно предсказанными вершиной и связью - {x[2]}
4. UAS - {x[3]}\n5. LAS - {x[4]}\n6. Количество использованных виртуальных узлов в gold - {x[5]}\n7. Кол-во использованных виртуальных узлов в predicted - {x[6]}
8. Кол-во правильно предсказанных использованных виртуальных узлов (unlabeled) - {x[7]}\n9. Кол-во правильно предсказанных использованных виртуальных узлов (labeled) - {x[8]}
10. unlabeled precision по использованным виртуальным узлам - {x[9]}\n11. unlabeled recall - {x[10]}\n12. labeled precision - {x[11]}\n13. labeled recall - {x[12]}'''

In [15]:
%%time
print(pretty_result(evaluate(r'D:\maltParser\maltparser-1.9.1\examples\data\talbanken05_test.conllu', r'D:\maltParser\maltparser-1.9.1\out.conll')))

KeyboardInterrupt: 

In [4]:
%%time
print(pretty_result(evaluate(r'D:\maltParser\dist\dist-20141005\lib\gold.conll', r'D:\maltParser\dist\dist-20141005\lib\parse.conll')))

1. Всего слов - 157989
2. Кол-во слов с правильно предсказанной вершиной - 96640
3. Кол-во слов с правильно предсказанными вершиной и связью - 84560
4. UAS - 0.6116881555044972
5. LAS - 0.535227136066435
6. Количество использованных виртуальных узлов в gold - 0
7. Кол-во использованных виртуальных узлов в predicted - 0
8. Кол-во правильно предсказанных использованных виртуальных узлов (unlabeled) - 0
9. Кол-во правильно предсказанных использованных виртуальных узлов (labeled) - 0
10. unlabeled precision по использованным виртуальным узлам - 0
11. unlabeled recall - 0
12. labeled precision - 0
13. labeled recall - 0
CPU times: total: 359 ms
Wall time: 380 ms


In [5]:
%%time
print(pretty_result(evaluate(r'D:\Ready_Corpus\test.conll', r'D:\Ready_Corpus\test.conll.predicted')))

1. Всего слов - 172540
2. Кол-во слов с правильно предсказанной вершиной - 125721
3. Кол-во слов с правильно предсказанными вершиной и связью - 106160
4. UAS - 0.728648429349716
5. LAS - 0.6152776167845138
6. Количество использованных виртуальных узлов в gold - 8791
7. Кол-во использованных виртуальных узлов в predicted - 6348
8. Кол-во правильно предсказанных использованных виртуальных узлов (unlabeled) - 3850
9. Кол-во правильно предсказанных использованных виртуальных узлов (labeled) - 3455
10. unlabeled precision по использованным виртуальным узлам - 0.6064902331442974
11. unlabeled recall - 0.437947901262655
12. labeled precision - 0.5442659105229993
13. labeled recall - 0.39301558412012283
CPU times: total: 1.55 s
Wall time: 1.6 s


- TurboParser
    - Train size -  10
    - Test size -  5
    - Time spent for training:
        ~11h

    - train:
        2009\Gadzhety-neudachniki.tgt, 2010\Sokrashchenie_vuzov.tgt, 2003\Zemlya_naiznanku.tgt, 2003\Metis.tgt, 2009\Obrazovatelnoe_kreditovanie.tgt, 2013\Preodolet_kompleks_liliputa.tgt,
        2007\Sluchai_dlya_zhyuri.tgt, Uppsala\Korp_261.tgt, 2009\Reforma_obrazovaniya.tgt, 2010\Khudozhestvennaya_gimnastika.tgt
        Counter({'virt_V': 2203, 'virt_S': 1789, 'virt_A': 314, 'virt_ADV': 116, 'virt_CONJ': 83, 'virt_PR': 74, 'virt_PART': 34, 'virt_NUM': 26})

    - test: 
        test:
            news\Ya_35.tgt, 2006\Misha.tgt, 2017\Poet_i_prezident.tgt, 2020_RFFI\Ognev.tgt, 2009\Sanktsii.tgt
            Counter({'virt_V': 4611, 'virt_S': 2707, 'virt_A': 580, 'virt_ADV': 296, 'virt_PR': 200, 'virt_CONJ': 107, 'virt_PART': 30, 'virt_NUM': 24, 'virt_NID': 3})

    - Base creation time
        CPU times: total: 36.8 s
        Wall time: 39.7 s

    - Result:
        1. Всего слов - 172540
        2. Кол-во слов с правильно предсказанной вершиной - 125721
        3. Кол-во слов с правильно предсказанными вершиной и связью - 106160
        4. UAS - 0.728648429349716
        5. LAS - 0.6152776167845138
        6. Количество использованных виртуальных узлов в gold - 8791
        7. Кол-во использованных виртуальных узлов в predicted - 6348
        8. Кол-во правильно предсказанных использованных виртуальных узлов (unlabeled) - 3850
        9. Кол-во правильно предсказанных использованных виртуальных узлов (labeled) - 3455
        10. unlabeled precision по использованным виртуальным узлам - 0.6064902331442974
        11. unlabeled recall - 0.437947901262655
        12. labeled precision - 0.5442659105229993
        13. labeled recall - 0.39301558412012283

    - validation time:
        CPU times: total: 1.55 s
        Wall time: 1.6 s
 
- MaltParser (без смены разметки)
    - Train size -  10
    - Test size -  5
    - Time spent for trainig:
        ~5-10m

    - train: 
        news\Ya_13.tgt, news\Ya_31.tgt, 2011\Sinkhronnoe_plavanie.tgt, 2010\Kovcheg_Zhana_Vanie.tgt, 2011\Otkrytoe_pismo.tgt, 2010\Rodnye_i_priemnye.tgt, news\Ya_49.tgt, 
        2007\Chuvstvo_spravedlivosti.tgt, Uppsala\Korp_604.tgt, news\Ya_77.tgt
        Counter({'virt_V': 2285, 'virt_S': 2017, 'virt_A': 390, 'virt_PR': 79, 'virt_ADV': 60, 'virt_CONJ': 42, 'virt_PART': 16, 'virt_NUM': 14})

    - test:
        2003\Nobelevskie_premii.tgt, 2018\Poseshchenie_muzeya.tgt, 2006\Demonstratsiya_flaga.tgt, 2013\Sverkhnovaya_ekonomika.tgt, 2003\Uteshenie.tgt
        Counter({'virt_V': 1806, 'virt_S': 1397, 'virt_A': 232, 'virt_ADV': 70, 'virt_PR': 53, 'virt_CONJ': 35, 'virt_PART': 15, 'virt_NUM': 9})
    
    - base creation time
        CPU times: total: 22.9 s
        Wall time: 25.5 s
        
    - Result:
        1. Всего слов - 104351
        2. Кол-во слов с правильно предсказанной вершиной - 41840
        3. Кол-во слов с правильно предсказанными вершиной и связью - 31517
        4. UAS - 0.4009544709681747
        5. LAS - 0.3020287299594637
        6. Количество использованных виртуальных узлов в gold - 3717
        7. Кол-во использованных виртуальных узлов в predicted - 162
        8. Кол-во правильно предсказанных использованных виртуальных узлов (unlabeled) - 46
        9. Кол-во правильно предсказанных использованных виртуальных узлов (labeled) - 26
        10. unlabeled precision по использованным виртуальным узлам - 0.2839506172839506
        11. unlabeled recall - 0.012375571697605597
        12. labeled precision - 0.16049382716049382
        13. labeled recall - 0.006994888350820554
        
    - validation time
        CPU times: total: 688 ms
        Wall time: 693 ms
        
- MaltParser (UD разметка)
    - Time spent for training:
        ~15-20m

    - Result:
        1. Всего слов - 157989
        2. Кол-во слов с правильно предсказанной вершиной - 96640
        3. Кол-во слов с правильно предсказанными вершиной и связью - 84560
        4. UAS - 0.6116881555044972
        5. LAS - 0.535227136066435
        6. Количество использованных виртуальных узлов в gold - 0
        7. Кол-во использованных виртуальных узлов в predicted - 0
        8. Кол-во правильно предсказанных использованных виртуальных узлов (unlabeled) - 0
        9. Кол-во правильно предсказанных использованных виртуальных узлов (labeled) - 0
        10. unlabeled precision по использованным виртуальным узлам - 0
        11. unlabeled recall - 0
        12. labeled precision - 0
        13. labeled recall - 0

    - validation timr
        CPU times: total: 359 ms
        Wall time: 380 ms

